In [14]:
import numpy as np
import cupy as cp
import skimage.io
import skimage.color
import matplotlib.pyplot as plt
from skimage.transform import downscale_local_mean
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from numba import njit
import time


In [15]:
import pandas as pd
import numpy as np

def salvar_resultados_em_xlsx(lista_dicionarios, caminho_arquivo='resultados.xlsx'):
    # Converte os dicionários para DataFrame
    df = pd.DataFrame(lista_dicionarios)
    
    # Converte todos os np.float64 para float padrão do Python
    df = df.applymap(lambda x: float(x) if isinstance(x, np.float64) else x)
    
    # Salva como arquivo Excel
    df.to_excel(caminho_arquivo, index=False)
    print(f'Arquivo salvo em: {caminho_arquivo}')

In [16]:
@njit
def mirror_cpu(A, f):
    n, m = A.shape
    nlin = n + 2*f
    ncol = m + 2*f
    B = np.zeros((nlin, ncol), dtype=A.dtype)

    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]

    return B


In [17]:
@njit
def NLM_fast_cpu(img, h, f, t):
    m, n = img.shape
    filtrada = np.zeros((m, n), dtype=img.dtype)
    img_n = mirror_cpu(img, f)
    for i in range(m):
        for j in range(n):
            im = i + f
            jn = j + f
            W1 = img_n[im-f:im+f+1, jn-f:jn+f+1]
            rmin = max(im-t, f)
            rmax = min(im+t, m+f-1)
            smin = max(jn-t, f)
            smax = min(jn+t, n+f-1)
            NL = 0.0
            Z = 0.0
            for r in range(rmin, rmax+1):
                for s in range(smin, smax+1):
                    W2 = img_n[r-f:r+f+1, s-f:s+f+1]
                    d2 = np.sum((W1 - W2)**2)
                    sij = np.exp(-d2/(h**2))
                    Z += sij
                    NL += sij * img_n[r, s]
            filtrada[i, j] = NL / Z
    return filtrada


In [18]:
def mirror_gpu(A, f):
    n, m = A.shape
    B = cp.zeros((n + 2*f, m + 2*f), dtype=A.dtype)
    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]
    return B


In [19]:
nlm_kernel_global_code = r'''
extern "C" __global__
void nlm_kernel_global(
    const float* img_n, float* output,
    int m, int n, int f, int t, float h, int padded_width
) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i >= m || j >= n)
        return;

    int im = i + f;
    int jm = j + f;

    float NL = 0.0f;
    float Z = 0.0f;

    for (int r = im - t; r <= im + t; ++r) {
        for (int s = jm - t; s <= jm + t; ++s) {
            float d2 = 0.0f;
            for (int u = -f; u <= f; ++u) {
                for (int v = -f; v <= f; ++v) {
                    int x1 = im + u;
                    int y1 = jm + v;
                    int x2 = r + u;
                    int y2 = s + v;

                    float val1 = img_n[x1 * padded_width + y1];
                    float val2 = img_n[x2 * padded_width + y2];
                    float diff = val1 - val2;
                    d2 += diff * diff;
                }
            }
            float weight = __expf(-d2 / (h * h));
            Z += weight;
            NL += weight * img_n[r * padded_width + s];
        }
    }

    output[i * n + j] = NL / Z;
}
'''



In [20]:
import sklearn.neighbors as sknn
import networkx as nx

def process_pixel(i, j, img_n, f, t, h, nn, m, n):
    im = i + f
    jn = j + f
    patch_central = img_n[im - f:im + f + 1, jn - f:jn + f + 1]
    central = patch_central.ravel()

    rmin = max(im - t, f)
    rmax = min(im + t, m + f)
    smin = max(jn - t, f)
    smax = min(jn + t, n + f)

    n_patches = (rmax - rmin) * (smax - smin)
    patch_size = (2 * f + 1) ** 2

    dataset = np.empty((n_patches, patch_size), dtype=np.float32)
    pixels_busca = np.empty(n_patches, dtype=np.float32)

    source = -1
    k = 0
    for r in range(rmin, rmax):
        for s in range(smin, smax):
            W = img_n[r - f:r + f + 1, s - f:s + f + 1]
            neighbor = W.ravel()
            dataset[k, :] = neighbor
            pixels_busca[k] = img_n[r, s]
            if np.array_equal(central, neighbor):
                source = k
            k += 1

    if source == -1:
        source = 0  # fallback para evitar erro

    knnGraph = sknn.kneighbors_graph(dataset, n_neighbors=nn, mode='distance')
    G = nx.from_scipy_sparse_array(knnGraph)
    length, _ = nx.single_source_dijkstra(G, source)

    points = list(length.keys())
    distancias = np.array(list(length.values()), dtype=np.float32)

    similaridades = np.exp(-distancias ** 2 / (h ** 2))
    pixels = pixels_busca[points]

    NL = np.sum(similaridades * pixels)
    Z = np.sum(similaridades)
    return NL / Z if Z > 0 else img_n[im, jn]


from joblib import Parallel, delayed

def Parallel_GEONLM(img_n, f, t, h, nn):
    print(f'img_n.shape: {img_n.shape}')
    m = img_n.shape[0] - 2 * f
    n = img_n.shape[1] - 2 * f
    print(f'M: {m}, N: {n}')

    filtrada = Parallel(n_jobs=-1)(
        delayed(process_pixel)(i, j, img_n, f, t, h, nn, m, n)
        for i in range(m)
        for j in range(n)
    )

    filtrada_geo = np.array(filtrada).reshape((m, n))
    return filtrada_geo

In [21]:
def NLM_fast_cuda_global(img, h, f, t):
    img = img.astype(cp.float32)
    m, n = img.shape
    padded = mirror_gpu(img, f)

    module = cp.RawModule(code=nlm_kernel_global_code, options=('-std=c++11',))
    kernel = module.get_function("nlm_kernel_global")

    output = cp.zeros((m, n), dtype=cp.float32)
    threads_per_block = (16, 16)
    grid = ((n + 15) // 16, (m + 15) // 16)

    kernel(
        grid, threads_per_block,
        (
            padded.ravel(), output.ravel(),
            cp.int32(m), cp.int32(n), cp.int32(f), cp.int32(t),
            cp.float32(h), cp.int32(padded.shape[1])
        )
    )
    return output

def add_noise_gaussian(img):
    m, n = img.shape
    sigma = 10
    noise = np.random.normal(0, sigma, (m, n)).astype(np.float32)
    noised = np.clip(img + noise, 0, 255)
    return noised

def add_poisson_noise(img):    
    poisson_img = np.random.poisson(img)    
    return poisson_img

def add_poisson_gaussian_noise(image, gaussian_sigma=25):
    # Apply Poisson noise
    poisson_image = np.random.poisson(image)

    # Apply Gaussian noise
    gaussian_noise = np.random.normal(loc=0.0, scale=gaussian_sigma, size=image.shape)
    noisy_image = poisson_image + gaussian_noise 

    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    
    return noisy_image

def add_salt_and_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):   
    """
    Adds salt and pepper noise to an image.

    Parameters:
    - image: input image (numpy array).
    - salt_prob: probability of salt pixels (white).
    - pepper_prob: probability of pepper pixels (black).

    Returns:
    - Image with salt and pepper noise.
    """
    
    row, col = image.shape
    noisy = np.copy(image)

    # Adds salt noise
    salt_pixels = np.random.rand(row, col) < salt_prob
    noisy[salt_pixels] = 255

    # Adds pepper noise
    pepper_pixels = np.random.rand(row, col) < pepper_prob
    noisy[pepper_pixels] = 0

    return noisy.astype(np.uint8)

def downscale(img):
    img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
    return img_downscale

def anscombe_transform(img):
    return 2.0 * np.sqrt(img.astype(np.float32) + 3.0 / 8.0)

def inverse_anscombe(y):
    # Inversa exata de Makitalo & Foi (IEEE T-IP 2011)
    return ((y / 2.0)**2) - 3.0 / 8.0

def compute_adaptive_q(sigma_est, anscombe=False):
    q_nlm = 0.8 + 0.5 * np.tanh(0.3 * (sigma_est - 1))
    q_geo = 1.0 + 0.7 * np.tanh(0.25 * (sigma_est - 1.5))

    if anscombe:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 10)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 10)
    else:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 100)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 100)

    print(f'q_nlm: {q_nlm}, g_geo: {q_geo}')
    return q_nlm, q_geo

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def select_best_h_using_adaptive_q(image, image_gpu, q_nlm_candidates, f, t, alpha=0.5, anscombe=False):
    melhor_score = -float('inf')
    melhor_q_nlm = None
    melhor_resultado = None
    melhor_psnr = None
    melhor_ssim = None

    for h_nlm in q_nlm_candidates:
        #print(f'h_nlm: {h_nlm}')

        # Aplica NLM
        result_gpu = NLM_fast_cuda_global(image_gpu, h_nlm, f, t)
        cp.cuda.Stream.null.synchronize()

        # Referência: imagem original
        img_ref = np.clip(image, 0, 255).astype(np.uint8)

        if anscombe:
            # Inversa de Anscombe
            result_processed = np.maximum(inverse_anscombe(cp.asnumpy(result_gpu)), 0)
        else:
            result_processed = cp.asnumpy(result_gpu)

        result_uint8 = np.clip(result_processed, 0, 255).astype(np.uint8)

        # Métricas
        psnr = peak_signal_noise_ratio(img_ref, result_uint8, data_range=255)
        ssim = structural_similarity(img_ref, result_uint8, data_range=255)

        # Score combinado
        score = alpha * psnr + (1 - alpha) * (ssim * 100)
        print(f"h = {h_nlm:.2f} | PSNR = {psnr:.2f} | SSIM = {ssim:.4f} | Score = {score:.2f}")

        if score > melhor_score:
            melhor_score = score
            melhor_q_nlm = h_nlm
            melhor_resultado = result_processed
            melhor_psnr = psnr
            melhor_ssim = ssim

    print(f"\n[SELECIONADO] H = {melhor_q_nlm:.2f} | PSNR = {melhor_psnr:.2f} | SSIM = {melhor_ssim:.4f} | SCORE = {melhor_score:.2f}")

    return melhor_resultado, melhor_q_nlm, melhor_psnr, melhor_ssim, melhor_score


In [22]:
import os
def read_directories(directory, img=None, exclude_json=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)
        elif exclude_json is not None:
            filenames.append(filename.replace('.json',''))     
        else:
            filenames.append(filename)    
    return filenames

### Noise Salt

In [23]:
from skimage.restoration import estimate_sigma
import sys


from pathlib import Path


dir_images = ("/workspace/ProjetoDoutorado/wvc/images")



#dir_out_nlm = f'{root_dir}/wvc/best_out_put_Gaussian_NLM'
#dir_out_geonlm = f'{root_dir}/wvc/best_out_put_Gaussian_GEONLM'


array_dir = read_directories(dir_images)

array_nln_salt_256_filtereds = []

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images}/{file_name}')
    img = img[0, :, :] if len(img.shape) > 2 else img
    if len(img.shape) > 2:
        img = skimage.color.rgb2gray(img)
        img = 255 * img

    img_downscale = downscale(img)
    m, n = img.shape
    ruidosa = add_salt_and_pepper_noise(img_downscale)

    # Clipa imagem para intervalo [0, 255]
    ruidosa[np.where(ruidosa > 255)] = 255
    ruidosa[np.where(ruidosa < 0)] = 0

    # Preparo para CPU e GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)
    img_gpu_noisse_gaussian = cp.array(img_noisse_gaussian_np)


    h_nlm , g_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)

    q_nlm_candidates = np.array([h_nlm + delta for delta in range(-50, 25, 1)])


    img_filtered_nlm, nlm_h, nlm_psnr, nlm_ssim, nlm_score = select_best_h_using_adaptive_q(
        image=img_downscale,
        image_gpu=img_gpu_noisse_gaussian,
        q_nlm_candidates=q_nlm_candidates,
        f=4, t=7,
        alpha=0.5,
        anscombe=False
    )

    dict = {
        'img_noisse_gaussian_np':img_noisse_gaussian_np,
        'img_filtered_nlm':img_filtered_nlm,
        'nlm_h':nlm_h, 
        'nlm_psnr': nlm_psnr,
        'nlm_ssim':nlm_ssim,
        'nlm_score':nlm_score,
        'file_name':file_name,
    }
    array_nln_salt_256_filtereds.append(dict)

q_nlm: 125, g_geo: 157
h = 75.00 | PSNR = 21.38 | SSIM = 0.7164 | Score = 46.51
h = 76.00 | PSNR = 21.38 | SSIM = 0.7163 | Score = 46.50
h = 77.00 | PSNR = 21.38 | SSIM = 0.7161 | Score = 46.50
h = 78.00 | PSNR = 21.39 | SSIM = 0.7159 | Score = 46.49
h = 79.00 | PSNR = 21.39 | SSIM = 0.7156 | Score = 46.48
h = 80.00 | PSNR = 21.39 | SSIM = 0.7154 | Score = 46.47
h = 81.00 | PSNR = 21.39 | SSIM = 0.7152 | Score = 46.46
h = 82.00 | PSNR = 21.40 | SSIM = 0.7150 | Score = 46.45
h = 83.00 | PSNR = 21.40 | SSIM = 0.7148 | Score = 46.44
h = 84.00 | PSNR = 21.40 | SSIM = 0.7146 | Score = 46.43
h = 85.00 | PSNR = 21.41 | SSIM = 0.7144 | Score = 46.42
h = 86.00 | PSNR = 21.41 | SSIM = 0.7142 | Score = 46.41
h = 87.00 | PSNR = 21.42 | SSIM = 0.7139 | Score = 46.40
h = 88.00 | PSNR = 21.42 | SSIM = 0.7137 | Score = 46.39
h = 89.00 | PSNR = 21.42 | SSIM = 0.7135 | Score = 46.39
h = 90.00 | PSNR = 21.43 | SSIM = 0.7132 | Score = 46.37
h = 91.00 | PSNR = 21.43 | SSIM = 0.7130 | Score = 46.37
h = 92.0

In [24]:
import pickle
from pathlib import Path

def save_pickle(obj, path):
    path = Path(path)
    with path.open("wb") as f:
        # usa o protocolo mais moderno disponível
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(path):
    path = Path(path)
    with path.open("rb") as f:
        return pickle.load(f)


In [25]:
save_pickle(array_nln_salt_256_filtereds, "resultados_salt_256.pkl")

### Novo 09/09/25

In [26]:
def run_geonlm_pipeline(img_original, h_base, img_noisy, f, t, mult, nn=10):

    img_noisy_mirror = mirror_cpu(img_noisy, f)    
   
    img_n_geo = np.pad(img_noisy_mirror, ((f, f), (f, f)), 'symmetric')  
   
   
    h_geo = (h_base) * mult
    print(f"\nExecutando GEONLM com h = {h_geo:.2f} (base {h_base} * {mult})")

    img_geo = Parallel_GEONLM(img_n_geo, f=f, t=t, h=h_geo, nn=nn)

    img_geo_no_pad = img_geo[f:-f, f:-f]  # Remove 'f' pixels de cada lado
    img_geo_no_pad = np.clip(img_geo_no_pad, 0, 255).astype(np.uint8)

    img_ref = np.clip(img_original, 0, 255).astype(np.uint8)   

    psnr = peak_signal_noise_ratio(img_ref, img_geo_no_pad, data_range=255)
    ssim = structural_similarity(img_ref, img_geo_no_pad, data_range=255)

    score = 0.5 * psnr + 0.5 * (ssim * 100)
    print(f"→ PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, Score: {score:.2f}")    

   
    return img_geo_no_pad, h_geo, psnr, ssim, score

### Novo

In [28]:
vetor = load_pickle("resultados_salt_256.pkl")
from bm3d import bm3d, BM3DProfile
from skimage.restoration import estimate_sigma
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.morphology import disk


array_gnlm_bm3d_median_256_filtereds = []
for array in vetor:

    img_noisse_gaussian_np =array['img_noisse_gaussian_np']
    
    nlm_h =array['nlm_h']
    file_name =array['file_name']
    nlm_psnr =array['nlm_psnr']
    nlm_ssim =array['nlm_ssim']
    nlm_score =array['nlm_score']   

    dir_images = ("/workspace/ProjetoDoutorado/wvc/images")


    img = skimage.io.imread(f'{dir_images}/{file_name}')
    img = img[0, :, :] if len(img.shape) > 2 else img
    if len(img.shape) > 2:
        img = skimage.color.rgb2gray(img)
        img = 255 * img

    img_downscale = downscale(img)

    # Parâmetros
    f = 4
    t = 7
    nn = 10

    ini = time.time()

    print("\n--- Executando Pipeline 512x512 com mult = 1.55 ---")
    mult = 1.55
    img_filtered_gnlm, h_gnlm, psnr_gnlm, ssim_gnlm, score_gnlm = run_geonlm_pipeline(img_downscale, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   
    end = time.time()
    time_geonlm = end - ini

    img_bm3d = img_downscale.copy()

    if len(img_bm3d.shape) > 2:
        img_bm3d = skimage.color.rgb2gray(img_bm3d)
        img_bm3d = 255 * img_bm3d
    # Remove frames extras, se existirem (e.g. GIF com shape (1, H, W) ou (N, H, W, 3))
    if img_bm3d.ndim == 4:
        img_bm3d = img_bm3d[0]
    elif img_bm3d.ndim == 3 and img_bm3d.shape[2] != 3:
        img_bm3d = np.squeeze(img_bm3d)

    # Converte para escala de cinza, se necessário
    if img_bm3d.ndim == 3 and img_bm3d.shape[2] == 3:
        img_bm3d_gray = rgb2gray(img_bm3d)  # retorna float64 em [0, 1]
    else:
        img_bm3d_gray = img_bm3d.astype(np.float32) / 255.0  # já está em cinza
     # Agora está garantidamente em escala de cinza float [0,1]
    img_bm3d_gray = np.clip(img_bm3d_gray * 255, 0, 255).astype(np.uint8)
    
     # 1. Normaliza a imagem ruidosa para intervalo [0, 1]
    ruidosa_normalizada = img_noisse_gaussian_np.astype(np.float32) / 255.0

    sigma_est = estimate_sigma(ruidosa_normalizada, channel_axis=None)

    inicio = time.time()
    # 3. Define o perfil padrão do BM3D
    perfil_bm3d = BM3DProfile()
    fim = time.time() 
    time_bm3d = fim - inicio

    # 4. Aplica o BM3D com os argumentos definidos
    denoised = bm3d(
        ruidosa_normalizada,
        sigma_psd=sigma_est,
        profile=perfil_bm3d
    )
    denoised_sq = np.squeeze(denoised)

    
    psnr_bm3d = peak_signal_noise_ratio(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    ssim_bm3d = structural_similarity(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    score_bm3d = 0.5 * psnr_bm3d + 0.5 * (ssim_bm3d * 100)


    img_noisse_gaussian_np_median = img_noisse_gaussian_np.copy()
     # Clipa imagem para intervalo [0, 255]
    img_noisse_gaussian_np_median[np.where(img_noisse_gaussian_np_median > 255)] = 255
    img_noisse_gaussian_np_median[np.where(img_noisse_gaussian_np_median < 0)] = 0

    inicio = time.time()
    median_filter = median(img_noisse_gaussian_np_median, footprint=disk(3))    
    fim = time.time()    
    
    psnr_median = peak_signal_noise_ratio(img_downscale, median_filter,  data_range=255)
   
    ssim_median = structural_similarity(img_downscale, median_filter,  data_range=255)

    score_median = 0.5 * psnr_median + 0.5 * (ssim_median * 100)
    
    time_median = fim - inicio
   

    dict = {
        'img_filtered_gnlm':img_filtered_gnlm,
        'h_gnlm':h_gnlm,
        'psnr_gnlm':psnr_gnlm,
        'ssim_gnlm':ssim_gnlm,
        'score_gnlm':score_gnlm,  
        'time_geonlm':time_geonlm,

        'nlm_h':nlm_h, 
        'nlm_psnr': nlm_psnr,
        'nlm_ssim':nlm_ssim,
        'nlm_score':nlm_score,

        'psnr_median': psnr_median,
        'ssim_median': ssim_median,
        'score_median': score_median,
        'time_median':time_median,

        'psnr_bm3d': psnr_bm3d,
        'ssim_bm3d': ssim_bm3d,
        'score_bm3d': score_bm3d,
        'time_bm3d':time_bm3d,
        'file_name':file_name,

    }
    array_gnlm_bm3d_median_256_filtereds.append(dict)
save_pickle(array_gnlm_bm3d_median_256_filtereds, "resultados_salt_gnlm_bm3d_median_256.pkl")


--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 230.95 (base 149 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 24.96, SSIM: 0.7493, Score: 49.94

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 193.75 (base 125 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 27.52, SSIM: 0.7607, Score: 51.80

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 224.75 (base 145 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


→ PSNR: 26.71, SSIM: 0.6839, Score: 47.55

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 212.35 (base 137 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 27.14, SSIM: 0.7602, Score: 51.58

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 237.15 (base 153 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 24.86, SSIM: 0.7749, Score: 51.17

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 235.60 (base 152 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 24.52, SSIM: 0.8268, Score: 53.60

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 213.90 (base 138 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 28.58, SSIM: 0.7092, Score: 49.75

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 203.05 (base 131 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 27.55, SSIM: 0.8118, Score: 54.36

--- Executando Pipeline 512

In [29]:
salvar_resultados_em_xlsx(array_gnlm_bm3d_median_256_filtereds, 'saida_gnlm_bm3d_median_256_filtereds.xlsx')

Arquivo salvo em: saida_gnlm_bm3d_median_256_filtereds.xlsx


/tmp/ipykernel_56870/1866010612.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x) if isinstance(x, np.float64) else x)
